In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset

In [2]:
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
from PIL import Image
from os import listdir
from os.path import join

class SIGNSDataset(Dataset):
    def __init__(self, data_dir):
        # Store the paths to the image files and their corresponding labels
        labels = listdir(data_dir)
        self.filenames = []
        self.labels = []

        for i, label in enumerate(labels):
            label_dir = join(data_dir, label)
            for filename in listdir(label_dir):
                file_path = join(label_dir, filename)
                self.filenames.append(file_path)
                self.labels.append(i)
        
        self.transform = transforms.Compose([
            transforms.Resize(32),              
            transforms.RandomHorizontalFlip(),  
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ]) 

    def __len__(self):
        # Return the size of the dataset
        return len(self.filenames)
    
    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = Image.open(img_path)
        image = self.transform(image)
        label = self.labels[idx]
        return image, label

In [4]:
train_dataset = SIGNSDataset('Train_Dataset')

In [5]:
batch_size = 64

In [6]:
train_loader = DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle = True)

In [7]:
test_dataset = SIGNSDataset('Test_Dataset')

In [8]:
test_loader = DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle = True)

In [9]:
class NN(nn.Module):
    def __init__(self,num_classes):
    
        super(NN,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*13*13,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,num_classes)

    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)) , 2)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        output = self.fc3(x)
        return output

    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features =  num_features * s
        return num_features

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
num_classes = 10
learning_rate = 0.001
num_epochs = 10

In [12]:
model = NN(num_classes = num_classes).to(device)

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [23]:
for epoch in range(num_epochs):
    for batch_idx, (data,target) in enumerate(train_loader):
        data = data.to(device = device)
        targets = target.to(device = device)
        scores = model(data)
        loss = criterion(scores,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch == num_epochs-1:
        PATH = './Sign_net_' + str(epoch+1) + '.pth'
        torch.save(model.state_dict(),PATH)
        

In [24]:
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            
            model.load_state_dict(torch.load(f'Sign_net_{num_epochs}.pth', weights_only=True))
            model.to(device = device)
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    return num_correct,num_samples

In [25]:
num_correct,num_samples = check_accuracy(test_loader,model)
print((int(num_correct)/num_samples)*100)

85.94377510040161
